In [1]:
search_for = 138
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.09270429611206055
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 1097396311
type: [1, 1, 1, 1, 138] 138
cases of this type: 362673936
100000 45.48930586060579
200000 93.16697821929377
300000 103.95019120906245
400000 102.71352488856155
500000 102.86703158539673
600000 102.36702708451877
700000 105.56806454381561
800000 100.86331751311324
900000 100.3492293789726
1000000 99.50131139161437
1100000 100.08337376846556
1200000 99.75342723071832
1300000 81.58908633399128
1400000 40.112672714775826
1500000 99.19856334553651
1600000 98.86829166626754
1700000 99.23412522440314
1800000 98.06669927420178
1900000 99.12142666201368
2000000 104.7204238297004
2100000 100.62130097757911
2200000 99.67557985438985
2300000 98.59612980580134
2400000 98.6029108139159
2500000 99.20480313794367
2600000 94.99227630411445
2700000 23.923650458370826
2800000 94.71059512525088
2900000 94.80818764807636
3000000 94.357358

29400000 106.75287746555155
29500000 107.6940111407688
29600000 103.79210570613608
29700000 106.24435005887246
29800000 106.36041660557892
29900000 104.89357815653037
30000000 103.46785713393905
30100000 101.34601955456186
30200000 91.59963947008451
30300000 36.15224985784015
30400000 98.49948140127327
30500000 101.32001300004777
30600000 101.76660111738533
30700000 106.30152730886167
30800000 101.17507067976793
30900000 101.22541733766738
31000000 104.24825405355676
31100000 108.34583386594393
31200000 104.56587194498928
31300000 99.2119769110651
31400000 45.57853373303896
31500000 108.50552175039303
31600000 111.54619293011514
31700000 116.6154008630347
31800000 52.02817714090428
31900000 95.16440204274126
32000000 109.39418868832435
32100000 108.5368710624701
32200000 108.98487418825079
32300000 105.16197674788847
32400000 105.04885371697537
32500000 106.91459960722176
32600000 103.6712757134626
32700000 105.03128821828166
32800000 101.53497715044739
32900000 24.82162180548349
33000

59200000 24.063754058588295
59300000 99.59852391008202
59400000 98.46337804370874
59500000 96.79872889960524
59600000 98.83708094966967
59700000 102.50294976851244
59800000 100.67964389732819
59900000 102.80531362499671
60000000 100.60250007111914
60100000 43.986105593448244
60200000 108.34824370990343
60300000 105.37659121167289
60400000 106.65899008016996
60500000 104.69894955038778
60600000 105.56283965513828
60700000 105.17948771552389
60800000 103.28773731843756
60900000 66.58359904692813
61000000 79.12562499307388
61100000 101.81319059644375
61200000 97.30608899321197
61300000 97.358521691176
61400000 103.48974457480804
61500000 94.73616757382317
61600000 93.0942836894732
61700000 91.37059463240288
61800000 33.34505312603254
61900000 86.21405651914263
62000000 92.87987532415882
62100000 90.68865616456232
62200000 96.6450193899801
62300000 94.45595952088419
62400000 96.35607698907293
62500000 99.35110461093775
62600000 103.38158532104363
62700000 35.50650362519682
62800000 106.983

89300000 106.4985711563281
89400000 109.74846071155751
89500000 109.67139931539097
89600000 105.24047848505889
89700000 105.89321632558102
89800000 106.35299351478446
89900000 102.14183144768907
90000000 76.2826809323083
90100000 56.09258888580032
90200000 95.30718177682044
90300000 93.91909286784261
90400000 93.16250350428837
90500000 96.29768573193083
90600000 93.67157205426327
90700000 40.86589367449795
90800000 80.41404355160999
90900000 95.02161719864917
91000000 95.60118519839173
91100000 94.06923050024301
91200000 94.73613430255601
91300000 94.17687498105511
91400000 34.948751868602
91500000 103.84821806924585
91600000 101.85263684551899
91700000 104.0867028304667
91800000 103.21950708737299
91900000 105.26723681639993
92000000 110.94280907171266
92100000 103.71471219392056
92200000 100.7233020089917
92300000 101.94078440391839
92400000 99.71955280055234
92500000 100.50140408740651
92600000 99.14952304711474
92700000 31.92711474164635
92800000 92.41628761763364
92900000 92.17549

118900000 88.0914475442573
119000000 87.37130441473819
119100000 81.45193276031462
119200000 39.23402294442878
119300000 86.14414261231983
119400000 89.57578826266052
119500000 87.68923656887125
119600000 46.123440026318
119700000 69.25690168246638
119800000 87.04343642803758
119900000 86.42215662438231
120000000 89.76016776468488
120100000 33.5185361419553
120200000 91.52400421453494
120300000 87.58320941559128
120400000 90.42272280182718
120500000 92.97394554111989
120600000 94.10212096617327
120700000 98.73892491807847
120800000 94.41709536597008
120900000 98.0408008123848
121000000 94.41842996368842
121100000 99.74174367355806
121200000 100.8232817208313
121300000 98.88035970455788
121400000 93.95318290833019
121500000 92.73654693762356
121600000 90.81198825057405
121700000 88.51818944334869
121800000 44.81347928667761
121900000 74.75301842464587
122000000 89.1293849629238
122100000 87.51320670339211
122200000 67.95708599497003
122300000 46.60420155984349
122400000 89.1367896444200

...

982900000 7.4251248815994835
983000000 3.853612557040724
983100000 5.173405241057742
983200000 7.39018393831839
983300000 7.598266503209612
983400000 6.8756355204295
983500000 6.042920430261265
983600000 5.590562626220495
983700000 1.9822766223295143
983800000 5.768373234135059
983900000 6.004515108293473
984000000 6.57637844472716
984100000 5.836553297542238
984200000 5.730569136777625
984300000 3.502771827564157
984400000 2.86071705598198
984500000 6.11914666622928
984600000 5.751944813571382
984700000 6.447712677006561
984800000 5.892350916632035
984900000 5.283897027539413
985000000 2.8763788715326846
985100000 4.472728139478717
985200000 5.920447084983663
985300000 6.397993735887441
985400000 6.054367798168113
985500000 6.070194711174824
985600000 4.223397579734864
985700000 2.6874149051118335
985800000 5.729358505380503
985900000 5.9985596541887105
986000000 6.659814194637983
986100000 5.814286108030771
986200000 5.271570479742086
986300000 3.079587130672596
986400000 4.352417821

1011300000 0.8698355806137382
1011400000 2.786346379345264
1011500000 2.6711603202494247
1011600000 3.794036443882008
1011700000 2.852243019999319
1011800000 2.905655111799182
1011900000 1.432460235704856
1012000000 1.6423322595969558
1012100000 2.9326451311434893
1012200000 3.342618791851156
1012300000 3.0950379791032554
1012400000 2.5264335480411413
1012500000 2.2231577131640283
1012600000 0.8464342880197901
1012700000 2.2987030458497095
1012800000 2.8988071939892195
1012900000 2.7493194296219947
1013000000 2.5379039031467845
1013100000 2.3595275024113826
1013200000 1.4567259187808117
1013300000 1.1050704364985637
1013400000 2.951845633171878
1013500000 2.354227315890368
1013600000 2.771065034879681
1013700000 2.2281510527602206
1013800000 2.1660608065600275
1013900000 0.8595842096163652
1014000000 1.970052573228883
1014100000 2.402791615496971
1014200000 2.5143123539470413
1014300000 2.4028125432609855
1014400000 2.181357460990675
1014500000 1.5675269179847524
1014600000 0.893369538

1037200000 2.600472111435878
1037300000 3.2808799442879266
1037400000 3.6529453010497104
1037500000 2.9046431168126263
1037600000 3.313546470490872
1037700000 3.5382051640686556
1037800000 2.798695072833274
1037900000 3.495133570342287
1038000000 3.2401247036152445
1038100000 2.719147262920034
1038200000 3.278093644226808
1038300000 3.3252224177921548
1038400000 2.7490463347425145
1038500000 3.2674014466337113
1038600000 3.4111113313424126
1038700000 2.801412715766863
1038800000 3.2910016064140852
1038900000 3.5281143654102554
1039000000 3.0643349387031793
1039100000 3.0253927224835375
1039200000 3.734726624397492
1039300000 2.9748070580597146
1039400000 2.505500638118425
1039500000 3.8026461856561893
1039600000 3.3163430289560005
1039700000 2.325088601722551
1039800000 3.4502886551853287
1039900000 3.459735397291857
1040000000 2.928576591794864
1040100000 3.659750809746269
1040200000 4.376893042922827
best so far: 0
type: [1, 3, 1, 2, 23] 138
cases of this type: 6716184
1040300000 2.5

1058400000 1.5606436629050005
1058500000 2.1627792284331795
1058600000 1.831615798651911
1058700000 1.644602797079732
1058800000 1.925524058853257
1058900000 1.9246315773630247
1059000000 1.6333426004214189
1059100000 1.8945791270442665
1059200000 2.2664269347035755
1059300000 1.428192134903561
1059400000 1.6893286419563194
1059500000 2.2053995696155186
1059600000 1.7159312532318334
1059700000 1.4077835427805485
1059800000 2.05806279038545
1059900000 1.7852204289590323
1060000000 1.6419215611453555
1060100000 1.798037776539888
1060200000 2.087509971886114
1060300000 1.3440230749481081
1060400000 1.731056600344262
1060500000 2.0885467159396662
1060600000 1.5528418194830118
1060700000 1.3533836123702916
1060800000 1.9674627623284937
1060900000 1.652966480502492
1061000000 1.2986625826890712
1061100000 1.934247248088496
1061200000 1.9311828002509972
1061300000 1.2734897751815881
1061400000 1.6615378376079866
1061500000 1.9645755544955135
1061600000 1.2987058645978955
1061700000 1.52307516

1084900000 0.406564142913568
1085000000 0.3632336632836183
1085100000 0.3353919644043836
1085200000 0.320969618607682
1085300000 0.3075464679847393
1085400000 0.32886745808423096
1085500000 0.33216379074375035
1085600000 0.32819581372348866
1085700000 0.3162023771925231
1085800000 0.1902098529095471
1085900000 0.09328668822549978
1086000000 0.10065613246620496
1086100000 0.10492403131693005
1086200000 0.10204562395731542
1086300000 0.13797792195061248
1086400000 0.3800917544340213
1086500000 0.31376255056349966
1086600000 0.2890217392538819
1086700000 0.2880351893073678
1086800000 0.2899953602589753
1086900000 0.2868171518349535
1087000000 0.28006529965833427
1087100000 0.28518629067988727
1087200000 0.29115042547347875
best so far: 0
type: [2, 1, 3, 23, 1] 138
cases of this type: 109503
1087300000 0.2590978748528699
best so far: 0
type: [2, 1, 23, 1, 3] 138
cases of this type: 42849
1087400000 0.07470183518329065
best so far: 0
type: [2, 1, 23, 3, 1] 138
cases of this type: 14283
best